# Skin lesion classification of dermoscopic images using machine learning and convolutional neural network

19 December 2022

https://www.nature.com/articles/s41598-022-22644-9#Tab7

https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=561

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

In [3]:
def get_image_and_json_paths(src_path):
    image_paths = sorted(glob.glob(os.path.join(src_path, '**', '*.jpg'), recursive=True))
    json_paths = sorted(glob.glob(os.path.join(src_path, '**', '*.json'), recursive=True))

    return image_paths, json_paths

def get_image_data(image_paths):
    images = []

    for image_path in tqdm(image_paths, desc='Loading Images'):
        image = cv2.imread(image_path)
        if image is None:
            print(image_path)
        images.append(image)

    return images

def extract_metadata_and_locations_from_json(json_paths):
    meta_data, polygon_data, box_data = [], [], []
    for json_path in tqdm(json_paths, desc='Loading JSON', unit=' file'):
        try:
            with open(json_path, 'r', encoding='utf-8') as file:
                json_data = json.loads(re.sub(r'[\x00-\x1F\x7F-\x9F]', '', file.read()))
                labeling_info, metadata = json_data['labelingInfo'], json_data.get('metaData', None)

                filtered_metadata = {
                    'breed': metadata.get('breed', None),
                    'age': metadata.get('age', None),
                    'gender': metadata.get('gender', None),
                    'region': metadata.get('region', None),
                    'lesions': metadata.get('lesions', None)
                }

                meta_data.append(filtered_metadata)

                for entry in labeling_info:
                    if 'polygon' in entry:
                        polygon_data.extend(entry['polygon'].get('location', []))
                    if 'box' in entry:
                        box_data.extend(entry['box'].get('location', []))

        except Exception as e:
            print(json_path)
            print(e)

    return meta_data, polygon_data, box_data

def combine_images_and_masks(image_list, mask_list):
    combined_images = []

    for i in tqdm(range(len(image_list)), desc='Combining Images and Masks'):
        image = image_list[i]
        mask = mask_list[i]

        # Create an empty mask channel with the same shape as the image
        mask_channel = np.zeros_like(image[:, :, 0], dtype=np.uint8)

        # Set the mask channel to the mask values
        mask_channel[mask == 1] = 255

        # Stack the mask channel with the image
        combined_image = np.dstack((image, mask_channel))

        combined_images.append(combined_image)

    return combined_images

def resize_images(images, width, height):
    return np.array([cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA) for image in tqdm(images, desc='Resizing Images')])

def normalize_images(images):
    images = np.array(images, dtype=np.float32) / 255.0
    return images

def one_hot_encode_labels(labels):
    label_binarizer = LabelBinarizer()
    encoded_labels = label_binarizer.fit_transform(labels)
    return np.array(encoded_labels), label_binarizer

def create_polygon_binary_masks(image_list, polygon_data_list):
    binary_masks = []

    for image, polygon_data in tqdm(zip(image_list, polygon_data_list), desc='Generating Binary Masks', total=len(image_list)):
        mask = np.zeros(image.shape[:2], dtype=np.uint8)

        for poly_coords in polygon_data:
            if poly_coords:
                poly_points = []
                i = 1
                while f'x{i}' in poly_coords and f'y{i}' in poly_coords:
                    x = poly_coords[f'x{i}']
                    y = poly_coords[f'y{i}']
                    poly_points.append([x, y])
                    i += 1
                if len(poly_points) > 0:
                    poly_points = np.array(poly_points, dtype=np.int32)
                    cv2.fillPoly(mask, [poly_points], 255)
            print(poly_coords)

        binary_masks.append(mask)

    return binary_masks

def visualize_masks(image_data, mask_data, selected_indices, figsize=(10, 5)):
    for idx in selected_indices:
        image = image_data[idx]
        mask_map = mask_data[idx]

        plt.figure(figsize=figsize)
        plt.subplot(1, mask_map.shape[2] + 1, 1)
        plt.imshow(image)
        plt.title('Original Image')

        for ch in range(mask_map.shape[2]):
            plt.subplot(1, mask_map.shape[2] + 1, ch + 2)
            plt.imshow(mask_map[:, :, ch], cmap='gray')
            plt.title(f'Channel {ch}')

        plt.tight_layout()
        plt.show()

# camera cat

In [4]:
src_path = '/content/drive/Shareddrives/반려묘/일반카메라'

In [5]:
image_paths, json_paths = get_image_and_json_paths(src_path)

In [6]:
image_data = get_image_data(image_paths)

Loading Images: 100%|██████████| 5500/5500 [02:30<00:00, 36.64it/s]


In [7]:
meta_data, polygon_data, box_data = extract_metadata_and_locations_from_json(json_paths)

Loading JSON: 100%|██████████| 5500/5500 [00:03<00:00, 1440.34 file/s]


In [8]:
metadata_df = pd.DataFrame(meta_data)

In [9]:
metadata_df

,breed,age,gender,region,lesions
0,노르웨이숲,10,M,H,A7
1,노르웨이숲,10,M,B,A7
2,노르웨이숲,4,M,B,A7
3,노르웨이숲,10,F,B,A7
4,노르웨이숲,10,F,B,A7
...,...,...,...,...,...
5495,페르시안,2,F,L,A6
5496,코리안숏헤어,8,F,H,A6
5497,코리안숏헤어,5,M,H,A6
5498,코리안숏헤어,2,F,H,A6


In [10]:
resized_images = resize_images(image_data, 96, 96)
normalized_images = normalize_images(resized_images)

Resizing Images: 100%|██████████| 5500/5500 [00:39<00:00, 137.98it/s]


In [11]:
encoded_labels, label_binarizer = one_hot_encode_labels(metadata_df['lesions'])

In [12]:
train_images, test_images, train_labels, test_labels = train_test_split(resized_images, encoded_labels, test_size=0.2, random_state=42)
test_images, val_images, test_labels, val_labels = train_test_split(test_images, test_labels, test_size=0.5, random_state=42)

In [13]:
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Validation images shape:", val_images.shape)
print("Validation labels shape:", val_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

Train images shape: (4400, 96, 96, 3)
Train labels shape: (4400, 4)
Validation images shape: (550, 96, 96, 3)
Validation labels shape: (550, 4)
Test images shape: (550, 96, 96, 3)
Test labels shape: (550, 4)


In [14]:
model = models.Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(96,96,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(units=4, activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
history1 = model.fit(train_images, train_labels, epochs=150, batch_size=32, validation_data=(val_images, val_labels))

Epoch 1/150
138/138 [==============================] - 1s 8ms/step - loss: 0.1145 - accuracy: 0.9623 - val_loss: 2.1191 - val_accuracy: 0.6291
Epoch 2/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0856 - accuracy: 0.9714 - val_loss: 2.0188 - val_accuracy: 0.6636
Epoch 3/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0724 - accuracy: 0.9768 - val_loss: 1.9564 - val_accuracy: 0.6873
Epoch 4/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0868 - accuracy: 0.9725 - val_loss: 3.0220 - val_accuracy: 0.5891
Epoch 5/150
138/138 [==============================] - 1s 8ms/step - loss: 0.1007 - accuracy: 0.9709 - val_loss: 1.9071 - val_accuracy: 0.6836
Epoch 6/150
138/138 [==============================] - 1s 8ms/step - loss: 0.1111 - accuracy: 0.9657 - val_loss: 2.3627 - val_accuracy: 0.6418
Epoch 7/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0831 - accuracy: 0.9734 - val_loss: 2.0313 - val_accuracy: 0.6345

In [ ]:
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(test_labels, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

18/18 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.43      0.15      0.22        41
           1       0.82      0.63      0.71        99
           2       0.70      0.44      0.54       101
           3       0.72      0.93      0.81       309

    accuracy                           0.72       550
   macro avg       0.67      0.53      0.57       550
weighted avg       0.71      0.72      0.70       550



In [17]:
combined_df = pd.concat([pd.DataFrame(resized_images), metadata_df.drop(columns='lesions')], axis=1)

ValueError: ignored

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(combined_df, encoded_labels, test_size=0.2, random_state=42)
test_images, val_images, test_labels, val_labels = train_test_split(test_images, test_labels, test_size=0.5, random_state=42)

In [ ]:
history2 = model.fit(train_images, train_labels, epochs=150, batch_size=32, validation_data=(val_images, val_labels))

In [ ]:
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(test_labels, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

In [ ]:
polygon_binary_masks = create_polygon_binary_masks(image_data[:5], polygon_data[:5])

In [ ]:
def visualize_binary_masks(mask_list, selected_indices, figsize=(10, 5)):
    for idx in selected_indices:
        mask = mask_list[idx]

        plt.figure(figsize=figsize)
        plt.imshow(mask, cmap='gray')
        plt.title("Binary Mask")

        plt.tight_layout()
        plt.show()

# 예시 데이터
selected_indices = [0, 5, 10]  # 선택한 이미지 인덱스 리스트

# 시각화
visualize_binary_masks(polygon_binary_masks, selected_indices)

In [ ]:
polygon_data[:2]

In [ ]:
box_data[:2]

In [ ]:
polygon_binary_masks = [mask_map[..., 0] for mask_map in masks_maps]

In [ ]:
polygon_binary_masks = combine_images_and_masks(image_data, polygon_binary_masks)

In [ ]:
visualize_masked_images(image_data, polygon_binary_masks, selected_indices=[0, 5, 10])

In [ ]:
resized_images = resize_images(image_data, 96, 96)
normalized_images = normalize_images(resized_images)

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(resized_images, encoded_labels, test_size=0.3, random_state=42)
test_images, val_images, test_labels, val_labels = train_test_split(test_images, test_labels, test_size=0.5, random_state=42)

In [ ]:
history = model.fit(train_images, train_labels, epochs=150, batch_size=32, validation_data=(val_images, val_labels))

In [ ]:
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(test_labels, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

In [ ]:
polygon_segmentation_maps = [mask_map[..., 1] for mask_map in masks_maps]

In [ ]:
polygon_segmentation_maps = combine_images_and_masks(image_data, polygon_segmentation_maps)

In [ ]:
visualize_masked_images(image_data, polygon_segmentation_maps, selected_indices=[0, 5, 10])

In [ ]:
box_binary_masks = [mask_map[..., 2] for mask_map in masks_maps]

In [ ]:
box_binary_masks = combine_images_and_masks(image_data, box_binary_masks)

In [ ]:
visualize_masked_images(image_data, box_binary_masks, selected_indices=[0, 5, 10])

In [ ]:
box_segmentation_maps = [mask_map[..., 3] for mask_map in masks_maps]

In [ ]:
box_segmentation_maps = combine_images_and_masks(image_data, box_segmentation_maps)

In [ ]:
visualize_masked_images(image_data, box_segmentation_maps, selected_indices=[0, 5, 10])

In [ ]:
original_size_mask_maps = resize_images(masks_maps, 96, 96)

In [ ]:
resized_mask_maps = create_mask_maps(resize_images(image_data, 96, 96), polygon_data, box_data)

In [ ]:
# TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 변환된 모델을 파일로 저장
with open('your_model.tflite', 'wb') as f:
    f.write(tflite_model)

# microscope cat